# Task 2: Data Insights

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import seaborn as sb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

In [2]:
# Read data files into pandas dataframes
# The first row of each of the sheets are comments, not part of the data
df_transactions = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='Transactions', skiprows=1)
df_new_customer_list = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='NewCustomerList', skiprows=1)
df_customer_demographic = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerDemographic', skiprows=1)
df_customer_address = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerAddress', skiprows=1)

In [3]:
df_transactions.sample()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
7415,7416,9,827,2017-11-03,1.0,Approved,OHM Cycles,Road,medium,medium,742.54,667.4,33549.0


In [4]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [5]:
df_new_customer_list.sample()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
906,Conway,Juarez,Male,27,1967-03-02,Help Desk Technician,NaN,Affluent Customer,N,No,...,QLD,Australia,5,0.78,0.78,0.78,0.78,904,904,0.5


In [6]:
df_new_customer_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            971 non-null    object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   int64         
 4   DOB                                  983 non-null    datetime64[ns]
 5   job_title                            894 non-null    object        
 6   job_industry_category                835 non-null    object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure       

In [7]:
df_customer_demographic.sample()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
2611,2612,Bran,Sauven,Male,33,1969-04-30,Statistician I,Retail,Affluent Customer,N,é¨è½æ ¼,No,18.0


In [8]:
df_customer_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  default     

In [9]:
df_customer_address.sample()

,customer_id,address,postcode,state,country,property_valuation
1717,1722,86 Menomonie Plaza,2290,NSW,Australia,4


In [10]:
df_customer_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


## Combine New Customer List with Customer Data for Data Cleaning

In [11]:
df_new_customer_list.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.56,0.7000,0.8750,0.743750,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.89,0.8900,1.1125,0.945625,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.01,1.0100,1.0100,1.010000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.87,1.0875,1.0875,1.087500,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.52,0.5200,0.6500,0.650000,4,4,1.703125


In [12]:
df_new_customer_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            971 non-null    object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   int64         
 4   DOB                                  983 non-null    datetime64[ns]
 5   job_title                            894 non-null    object        
 6   job_industry_category                835 non-null    object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure       

In [13]:
# We have firve unnamed columns.
# By opening and reading the xlsx files, they are possibly garbage values.
# We will exclude them from our analysis.
df_new_customer_list.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'], axis = 1, inplace=True)

# The rank and value column does not relate to anything in the archive. 
# We simply remove the columns.
df_new_customer_list.drop(['Rank', 'Value'], axis = 1, inplace=True)

In [14]:
df_customer = df_customer_demographic.merge(df_customer_address, how='outer', on='customer_id')
df_customer

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0,060 Morning Avenue,2016.0,New South Wales,Australia,10.0
1,2,Eli,Bockman,Male,81.0,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0,6 Meadow Vale Court,2153.0,New South Wales,Australia,10.0
2,3,Arlin,Dearle,Male,61.0,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0,NaN,NaN,NaN,NaN,NaN
3,4,Talbot,NaN,Male,33.0,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0,0 Holy Cross Court,4211.0,QLD,Australia,9.0
4,5,Sheila-kathryn,Calton,Female,56.0,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0,17979 Del Mar Point,2448.0,New South Wales,Australia,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3998,3999,Patrizius,NaN,Male,11.0,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Â¡â¢Â£Â¢âÂ§Â¶â¢ÂªÂºââ,Yes,10.0,1482 Hauk Trail,3064.0,VIC,Australia,3.0
3999,4000,Kippy,Oldland,Male,76.0,1991-11-05,Software Engineer IV,NaN,Affluent Customer,N,0/0,No,11.0,57042 Village Green Point,4511.0,QLD,Australia,6.0
4000,4001,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87 Crescent Oaks Alley,2756.0,NSW,Australia,10.0
4001,4002,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8194 Lien Street,4032.0,QLD,Australia,7.0


In [15]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4003 entries, 0 to 4002
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4003 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   float64       
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  default     

In [16]:
# The default column does not relate to anything in the archive.
# We simply remove the column.
df_customer.drop(['default'], axis = 1, inplace=True)

In [17]:
df_customer['customer_id'].describe()

count    4003.000000
mean     2002.000000
std      1155.710892
min         1.000000
25%      1001.500000
50%      2002.000000
75%      3002.500000
max      4003.000000
Name: customer_id, dtype: float64

In [18]:
# Set new customers id=10000+ for distinguish
df_new_customer_list['customer_id'] = 10000+df_new_customer_list.index
df_new_customer_list.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,customer_id
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,10000
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,10001
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,10002
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,10003
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,10004


In [19]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4003 entries, 0 to 4002
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4003 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   float64       
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  owns_car    

In [20]:
df_new_customer_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            971 non-null    object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   int64         
 4   DOB                                  983 non-null    datetime64[ns]
 5   job_title                            894 non-null    object        
 6   job_industry_category                835 non-null    object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure       

In [21]:
# Combine data
df_combined = pd.concat([df_customer, df_new_customer_list])
df_combined

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,060 Morning Avenue,2016.0,New South Wales,Australia,10.0
1,2,Eli,Bockman,Male,81.0,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,6 Meadow Vale Court,2153.0,New South Wales,Australia,10.0
2,3,Arlin,Dearle,Male,61.0,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,NaN,NaN,NaN,NaN,NaN
3,4,Talbot,NaN,Male,33.0,1961-10-03,NaN,IT,Mass Customer,N,No,7.0,0 Holy Cross Court,4211.0,QLD,Australia,9.0
4,5,Sheila-kathryn,Calton,Female,56.0,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,8.0,17979 Del Mar Point,2448.0,New South Wales,Australia,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10995,Ferdinand,Romanetti,Male,60.0,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,9.0,2 Sloan Way,2200.0,NSW,Australia,7.0
996,10996,Burk,Wortley,Male,22.0,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6.0,04 Union Crossing,2196.0,NSW,Australia,10.0
997,10997,Melloney,Temby,Female,17.0,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,15.0,33475 Fair Oaks Junction,4702.0,QLD,Australia,2.0
998,10998,Dickie,Cubbini,Male,30.0,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,19.0,57666 Victoria Way,4215.0,QLD,Australia,2.0


## Data Cleaning

In [22]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5003 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          5003 non-null   int64         
 1   first_name                           5000 non-null   object        
 2   last_name                            4846 non-null   object        
 3   gender                               5000 non-null   object        
 4   past_3_years_bike_related_purchases  5000 non-null   float64       
 5   DOB                                  4896 non-null   datetime64[ns]
 6   job_title                            4388 non-null   object        
 7   job_industry_category                4179 non-null   object        
 8   wealth_segment                       5000 non-null   object        
 9   deceased_indicator                   5000 non-null   object        
 10  owns_car     

In [23]:
# Gender
df_combined['gender'].unique()

array(['F', 'Male', 'Female', 'U', 'Femal', 'M', nan], dtype=object)

In [24]:
values = {'Male': 'M', 'Female': 'F', 'Femal': 'F'}
df_combined.replace(to_replace=values, inplace=True)
df_combined['gender'].unique()

array(['F', 'M', 'U', nan], dtype=object)

In [25]:
df_combined[df_combined['gender'].isnull()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation
4000,4001,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,87 Crescent Oaks Alley,2756.0,NSW,Australia,10.0
4001,4002,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,8194 Lien Street,4032.0,QLD,Australia,7.0
4002,4003,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,320 Acker Drive,2251.0,NSW,Australia,7.0


In [26]:
# We simply remove the three entries as most of the data are Nan
df_combined = df_combined[df_combined['gender'].notnull()]

In [27]:
# We exclude names from analysis
df_combined.drop(['first_name', 'last_name'], axis = 1, inplace=True)

/Users/macbookpro/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [28]:
df_combined['past_3_years_bike_related_purchases'].unique()

array([93., 81., 61., 33., 56., 35.,  6., 31., 97., 49., 99., 58., 38.,
       85., 91., 57., 79., 76., 72., 74., 43., 55., 12., 37.,  5., 62.,
       18.,  3., 17., 44., 59., 40., 46., 64., 24., 63., 98., 51., 68.,
       22.,  2., 48., 87., 26., 60., 47., 73., 21., 67., 16., 78., 30.,
       94., 28., 20., 11.,  1., 75., 41., 69., 19., 80., 83., 25., 54.,
        4., 23., 65., 88., 10., 77.,  9., 82., 27., 53., 32., 34., 71.,
       36., 90., 95.,  8., 39.,  7., 42., 13., 45., 50., 14., 89., 84.,
       96., 70., 66.,  0., 15., 86., 92., 29., 52.])

In [29]:
# Convert DOB to age.
df_transactions['transaction_date'].sort_values()

516     2017-01-01
5876    2017-01-01
3459    2017-01-01
12484   2017-01-01
19130   2017-01-01
           ...    
605     2017-12-30
15269   2017-12-30
19906   2017-12-30
15756   2017-12-30
12003   2017-12-30
Name: transaction_date, Length: 20000, dtype: datetime64[ns]

In [30]:
# The data was collected in 2017. We convert age to 2017.
df_combined['DOB'] = pd.to_datetime(df_combined['DOB'], errors='coerce', format='%Y-%m-%d')
year_2017 = pd.Timestamp('2017-12-31')
df_combined['age'] = (year_2017 - df_combined['DOB']).astype('<m8[Y]')

# drop DOB
df_combined.drop(['DOB'], axis = 1, inplace=True)

df_combined.head()

<ipython-input-30-c708d06b6a20>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['DOB'] = pd.to_datetime(df_combined['DOB'], errors='coerce', format='%Y-%m-%d')
<ipython-input-30-c708d06b6a20>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['age'] = (year_2017 - df_combined['DOB']).astype('<m8[Y]')


,customer_id,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,age
0,1,F,93.0,Executive Secretary,Health,Mass Customer,N,Yes,11.0,060 Morning Avenue,2016.0,New South Wales,Australia,10.0,64.0
1,2,M,81.0,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,6 Meadow Vale Court,2153.0,New South Wales,Australia,10.0,37.0
2,3,M,61.0,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,NaN,NaN,NaN,NaN,NaN,63.0
3,4,M,33.0,NaN,IT,Mass Customer,N,No,7.0,0 Holy Cross Court,4211.0,QLD,Australia,9.0,56.0
4,5,F,56.0,Senior Editor,NaN,Affluent Customer,N,Yes,8.0,17979 Del Mar Point,2448.0,New South Wales,Australia,4.0,40.0


In [31]:
df_combined[df_combined.age.isnull()]

,customer_id,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,age
143,144,U,71.0,Environmental Tech,IT,Mass Customer,N,No,NaN,22 Oxford Junction,2315.0,New South Wales,Australia,3.0,NaN
167,168,U,8.0,General Manager,IT,Affluent Customer,N,Yes,NaN,16 Golf View Center,3020.0,VIC,Australia,6.0,NaN
266,267,U,53.0,NaN,IT,High Net Worth,N,No,NaN,48 Arizona Drive,2768.0,NSW,Australia,10.0,NaN
289,290,U,42.0,Senior Sales Associate,IT,Mass Customer,N,No,NaN,6923 Tennessee Circle,2766.0,NSW,Australia,9.0,NaN
450,451,U,37.0,Quality Control Specialist,IT,High Net Worth,N,No,NaN,8385 Lien Drive,3192.0,Victoria,Australia,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,10775,U,62.0,Paralegal,Financial Services,Affluent Customer,N,No,5.0,95960 Warner Parkway,3842.0,VIC,Australia,1.0,NaN
835,10835,U,88.0,General Manager,IT,Mass Customer,N,No,13.0,768 Southridge Drive,2112.0,NSW,Australia,11.0,NaN
883,10883,U,24.0,NaN,IT,Affluent Customer,N,No,2.0,01 Bunker Hill Drive,2230.0,NSW,Australia,10.0,NaN
904,10904,U,0.0,Legal Assistant,IT,Mass Customer,N,No,2.0,276 Anthes Court,2450.0,NSW,Australia,6.0,NaN


In [32]:
# The above new customers will be removed from our analysis for now
df_combined = df_combined[df_combined.age.notnull()]

In [33]:
# Drop categorical variables with more than 15 categories.
print('job_title: ' + str(len(df_combined['job_title'].unique())))
print('job_industry_category: ' + str(len(df_combined['job_industry_category'].unique())))
print('wealth_segment: ' + str(len(df_combined['wealth_segment'].unique())))

job_title: 196
job_industry_category: 10
wealth_segment: 3


In [34]:
# job_title need to be dropped
df_combined.drop(['job_title'], axis = 1, inplace=True)

# address and postcode should also be dropped
df_combined.drop(['address', 'postcode'], axis = 1, inplace=True)

In [35]:
df_combined['job_industry_category'].unique()

array(['Health', 'Financial Services', 'Property', 'IT', nan, 'Retail',
       'Argiculture', 'Manufacturing', 'Telecommunications',
       'Entertainment'], dtype=object)

In [36]:
df_combined[df_combined['job_industry_category'].isnull()].count()

customer_id                            821
gender                                 821
past_3_years_bike_related_purchases    821
job_industry_category                    0
wealth_segment                         821
deceased_indicator                     821
owns_car                               821
tenure                                 821
state                                  820
country                                820
property_valuation                     820
age                                    821
dtype: int64

In [37]:
# We have too many null values in this column.
# We can simply drop the column
df_combined.drop(['job_industry_category'], axis = 1, inplace=True)

In [38]:
df_combined.wealth_segment.unique()

array(['Mass Customer', 'Affluent Customer', 'High Net Worth'],
      dtype=object)

In [39]:
df_combined['deceased_indicator'].value_counts()

N    4894
Y       2
Name: deceased_indicator, dtype: int64

In [40]:
# Exclude deceased customers from analysis
df_combined = df_combined[df_combined['deceased_indicator'] == 'N']
df_combined['deceased_indicator'].value_counts()

N    4894
Name: deceased_indicator, dtype: int64

In [41]:
# Then since all of entires are alive, we drop the decease column.
df_combined.drop(['deceased_indicator'], axis = 1, inplace=True)

In [42]:
df_combined.owns_car.unique()

array(['Yes', 'No'], dtype=object)

In [43]:
df_combined.tenure.unique()

array([11., 16., 15.,  7.,  8., 13., 20.,  9.,  6.,  1., 18., 21., 12.,
       19., 14.,  4., 22.,  5., 17.,  2.,  3., 10.])

In [44]:
df_combined[df_combined.tenure.isnull()].count()
# TODO: Will be taken care of in feature engineering
# Remove nan tenure for now

customer_id                            0
gender                                 0
past_3_years_bike_related_purchases    0
wealth_segment                         0
owns_car                               0
tenure                                 0
state                                  0
country                                0
property_valuation                     0
age                                    0
dtype: int64

In [45]:
df_combined = df_combined[df_combined.tenure.notnull()]

In [46]:
df_combined.state.unique()

array(['New South Wales', nan, 'QLD', 'VIC', 'NSW', 'Victoria'],
      dtype=object)

In [47]:
values = {'New South Wales': 'NSW', 'Victoria': 'VIC'}
df_combined.replace(to_replace=values, inplace=True)
df_combined['state'].unique()

array(['NSW', nan, 'QLD', 'VIC'], dtype=object)

In [48]:
df_combined[df_combined['state'].isnull()]

,customer_id,gender,past_3_years_bike_related_purchases,wealth_segment,owns_car,tenure,state,country,property_valuation,age
2,3,M,61.0,Mass Customer,Yes,15.0,NaN,NaN,NaN,63.0
9,10,F,49.0,Mass Customer,Yes,20.0,NaN,NaN,NaN,29.0
21,22,F,79.0,Mass Customer,No,11.0,NaN,NaN,NaN,55.0
22,23,M,43.0,High Net Worth,Yes,1.0,NaN,NaN,NaN,22.0


In [49]:
# We can simply drop the 4 entries with null state
df_combined = df_combined[df_combined['state'].notnull()]
df_combined[df_combined['state'].isnull()]

,customer_id,gender,past_3_years_bike_related_purchases,wealth_segment,owns_car,tenure,state,country,property_valuation,age


In [50]:
# We simply drop the country column since the data only consist of Australian data.
df_combined.drop(['country'], axis = 1, inplace=True)

In [51]:
df_combined['property_valuation'].unique()

array([10.,  9.,  4., 12.,  8.,  6.,  7.,  3.,  5., 11.,  1.,  2.])

In [52]:
df_combined['property_valuation'].value_counts()

9.0     806
8.0     790
10.0    677
7.0     615
11.0    337
6.0     299
5.0     278
4.0     263
12.0    235
3.0     230
2.0     180
1.0     180
Name: property_valuation, dtype: int64

In [53]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4890 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          4890 non-null   int64  
 1   gender                               4890 non-null   object 
 2   past_3_years_bike_related_purchases  4890 non-null   float64
 3   wealth_segment                       4890 non-null   object 
 4   owns_car                             4890 non-null   object 
 5   tenure                               4890 non-null   float64
 6   state                                4890 non-null   object 
 7   property_valuation                   4890 non-null   float64
 8   age                                  4890 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 382.0+ KB


## One-Hot Encoding

In [54]:
# Get list of categorical variables
s = (df_combined.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['gender', 'wealth_segment', 'owns_car', 'state']


In [55]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
df_OH_cols = pd.DataFrame(OH_encoder.fit_transform(df_combined[object_cols]))

# One-hot encoding removed index; put it back
df_OH_cols.index = df_combined.index

# Remove categorical columns (will replace with one-hot encoding)
num_df_combined = df_combined.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
df_OH_combined = pd.concat([num_df_combined, df_OH_cols], axis=1)

df_OH_combined.sample(10)

,customer_id,past_3_years_bike_related_purchases,tenure,property_valuation,age,0,1,2,3,4,5,6,7,8,9,10
1953,1954,88.0,5.0,9.0,50.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
615,616,36.0,6.0,4.0,22.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3702,3703,37.0,9.0,7.0,44.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
143,10143,6.0,17.0,9.0,42.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3682,3683,67.0,11.0,9.0,60.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3195,3196,11.0,8.0,7.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3039,3040,87.0,5.0,9.0,51.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3681,3682,94.0,11.0,3.0,39.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2393,2394,31.0,7.0,7.0,22.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
887,10887,27.0,5.0,9.0,60.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


## Separate customer data with new customer data

In [56]:
df_customer = df_OH_combined[df_OH_combined['customer_id'] < 9999].set_index('customer_id')
df_new_customer = df_OH_combined[df_OH_combined['customer_id'] >= 9999].reset_index(drop=True)
# df_new_customer.drop(['customer_id'], axis = 1, inplace=True)

In [57]:
df_customer

,past_3_years_bike_related_purchases,tenure,property_valuation,age,0,1,2,3,4,5,6,7,8,9,10
customer_id,,,,,,,,,,,,,,,
1,93.0,11.0,10.0,64.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,81.0,16.0,10.0,37.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
4,33.0,7.0,9.0,56.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
5,56.0,8.0,4.0,40.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6,35.0,13.0,9.0,51.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,93.0,14.0,12.0,42.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3996,8.0,19.0,6.0,42.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3997,87.0,1.0,2.0,16.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [58]:
df_new_customer

,customer_id,past_3_years_bike_related_purchases,tenure,property_valuation,age,0,1,2,3,4,5,6,7,8,9,10
0,10000,86.0,14.0,6.0,60.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,10001,69.0,16.0,11.0,47.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,10002,10.0,10.0,5.0,43.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,10003,64.0,5.0,1.0,38.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,10004,34.0,19.0,9.0,52.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,10995,60.0,9.0,7.0,58.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
979,10996,22.0,6.0,10.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
980,10997,17.0,15.0,2.0,63.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
981,10998,30.0,19.0,2.0,65.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


## RFM

In [59]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [60]:
df_transactions.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

We will use the `list_price` for our RFM analysis. 

In [61]:
df_transactions.list_price.value_counts().sort_values().sort_index()

12.01      195
16.08        1
26.15        1
32.44        1
36.78        1
          ... 
2064.08      1
2076.81      1
2083.94    208
2086.07      1
2091.47    465
Name: list_price, Length: 296, dtype: int64

In [62]:
df_transactions['order_status'].value_counts()

Approved     19821
Cancelled      179
Name: order_status, dtype: int64

In [63]:
# Remove cancelled orders
df_transactions = df_transactions[df_transactions['order_status'] != 'Cancelled']
df_transactions['order_status'].value_counts()

Approved    19821
Name: order_status, dtype: int64

In [64]:
# We remove columns that unrelate to RFM
df_transactions = df_transactions[['customer_id', 'transaction_date', 'list_price']]
df_transactions

,customer_id,transaction_date,list_price
0,2950,2017-02-25,71.49
1,3120,2017-05-21,2091.47
2,402,2017-10-16,1793.43
3,3135,2017-08-31,1198.46
4,787,2017-10-01,1765.30
...,...,...,...
19995,1018,2017-06-24,2005.66
19996,127,2017-11-09,416.98
19997,2284,2017-04-14,1636.90
19998,2764,2017-07-03,227.88


In [65]:
# Recency
max_date = df_transactions['transaction_date'].max()

recency_features_df = df_transactions \
    [['customer_id', 'transaction_date']] \
    .groupby('customer_id') \
    .apply(
        lambda x: (x['transaction_date'].max() - max_date) / pd.to_timedelta(1, 'day')
    ) \
    .to_frame() \
    .set_axis(['recency'], axis=1)

recency_features_df

,recency
customer_id,
1,-7.0
2,-128.0
3,-102.0
4,-195.0
5,-16.0
...,...
3497,-52.0
3498,-127.0
3499,-51.0


In [66]:
# Frequency
frequency_features_df = df_transactions \
    [['customer_id', 'transaction_date']] \
    .groupby('customer_id') \
    .count() \
    .set_axis(['frequency'], axis=1)

frequency_features_df

,frequency
customer_id,
1,11
2,3
3,8
4,2
5,6
...,...
3497,3
3498,6
3499,7


In [67]:
# Monetary
price_features_df = df_transactions \
    .groupby('customer_id') \
    .aggregate(
        {
            'list_price': ['sum', 'mean']
        }
    ) \
    .set_axis(['price_sum', 'price_mean'], axis=1)

price_features_df

,price_sum,price_mean
customer_id,,
1,9084.45,825.859091
2,4149.07,1383.023333
3,9888.23,1236.028750
4,1047.72,523.860000
5,5903.20,983.866667
...,...,...
3497,3744.07,1248.023333
3498,5177.06,862.843333
3499,7673.48,1096.211429


In [68]:
# Combine Features
features_df = pd.concat(
    [recency_features_df, frequency_features_df, price_features_df], axis=1
)
features_df

,recency,frequency,price_sum,price_mean
customer_id,,,,
1,-7.0,11,9084.45,825.859091
2,-128.0,3,4149.07,1383.023333
3,-102.0,8,9888.23,1236.028750
4,-195.0,2,1047.72,523.860000
5,-16.0,6,5903.20,983.866667
...,...,...,...,...
3497,-52.0,3,3744.07,1248.023333
3498,-127.0,6,5177.06,862.843333
3499,-51.0,7,7673.48,1096.211429


In [69]:
# Calculate RFM Score
features_df[['recency', 'frequency', 'price_sum']].describe()

,recency,frequency,price_sum
count,3493.000000,3493.000000,3493.000000
mean,-61.230461,5.674492,6285.198022
std,57.971602,2.311440,2941.998917
min,-353.000000,1.000000,60.340000
25%,-86.000000,4.000000,4162.960000
50%,-44.000000,6.000000,5974.770000
75%,-17.000000,7.000000,8157.720000
max,0.000000,14.000000,19071.320000


In [70]:
for index, row in features_df.iterrows():
    if row['recency'] > -17:
        features_df.at[index, 'r_score'] = 4
    elif row['recency'] > -44:
        features_df.at[index, 'r_score'] = 3
    elif row['recency'] > -86:
        features_df.at[index, 'r_score'] = 2
    elif row['recency'] >= -353:
        features_df.at[index, 'r_score'] = 1
        
    if row['frequency'] > 7:
        features_df.at[index, 'f_score'] = 4
    elif row['frequency'] > 6:
        features_df.at[index, 'f_score'] = 3
    elif row['frequency'] > 4:
        features_df.at[index, 'f_score'] = 2
    elif row['frequency'] >= 1:
        features_df.at[index, 'f_score'] = 1
        
    if row['price_sum'] > 8157.72:
        features_df.at[index, 'm_score'] = 4
    elif row['price_sum'] > 5974.77:
        features_df.at[index, 'm_score'] = 3
    elif row['price_sum'] > 4162.96:
        features_df.at[index, 'm_score'] = 2
    elif row['price_sum'] >= 60.34:
        features_df.at[index, 'm_score'] = 1

In [71]:
df_customer = df_customer.merge(features_df, how='left', on='customer_id')
df_customer

,past_3_years_bike_related_purchases,tenure,property_valuation,age,0,1,2,3,4,5,...,8,9,10,recency,frequency,price_sum,price_mean,r_score,f_score,m_score
customer_id,,,,,,,,,,,,,,,,,,,,,
1,93.0,11.0,10.0,64.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,-7.0,11.0,9084.45,825.859091,4.0,4.0,4.0
2,81.0,16.0,10.0,37.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,-128.0,3.0,4149.07,1383.023333,1.0,1.0,1.0
4,33.0,7.0,9.0,56.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-195.0,2.0,1047.72,523.860000,1.0,1.0,1.0
5,56.0,8.0,4.0,40.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,-16.0,6.0,5903.20,983.866667,4.0,2.0,2.0
6,35.0,13.0,9.0,51.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,-64.0,5.0,5931.69,1186.338000,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,93.0,14.0,12.0,42.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3996,8.0,19.0,6.0,42.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3997,87.0,1.0,2.0,16.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_customer[df_customer['recency'].isnull()].count()

past_3_years_bike_related_purchases    496
tenure                                 496
property_valuation                     496
age                                    496
0                                      496
1                                      496
2                                      496
3                                      496
4                                      496
5                                      496
6                                      496
7                                      496
8                                      496
9                                      496
10                                     496
recency                                  0
frequency                                0
price_sum                                0
price_mean                               0
r_score                                  0
f_score                                  0
m_score                                  0
dtype: int64

In [73]:
df_customer['recency'] = df_customer['recency'].fillna(-999)

In [74]:
df_customer = df_customer.fillna(0)

In [75]:
df_customer.isnull().values.any()

False

## Machine Learning

In [76]:
features = df_customer.columns.tolist()
features.remove('recency')
features.remove('frequency')
features.remove('price_sum')
features.remove('price_mean')

features.remove('r_score')
features.remove('f_score')
features.remove('m_score')

features

['past_3_years_bike_related_purchases',
 'tenure',
 'property_valuation',
 'age',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10]

In [77]:
X = df_customer[features]
y_r = df_customer['r_score']
y_f = df_customer['f_score']
y_m = df_customer['m_score']

predict_X = df_new_customer[features]

In [78]:
xgb_reg_spec = XGBRegressor(
    objective="reg:squarederror",
    random_state=123)

xgb_reg_model = GridSearchCV(
    estimator=xgb_reg_spec,
    param_grid=dict(
        learning_rate = [0.001, 0.01, 0.1, 0.3, 0.5]
    ),
    scoring='neg_mean_absolute_error',
    refit = True,
    cv = 5
)

### R Estimate

In [79]:
xgb_reg_model.fit(X, y_r)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=123,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_param

In [80]:
xgb_reg_model.best_score_

-1.2317398986024712

In [81]:
xgb_reg_model.best_params_

{'learning_rate': 0.1}

In [82]:
xgb_reg_model.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=123,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [83]:
predictions_r = xgb_reg_model.predict(predict_X)

### F Estimate

In [84]:
xgb_reg_model.fit(X, y_f)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=123,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_param

In [85]:
xgb_reg_model.best_score_

-1.1481572000899067

In [86]:
xgb_reg_model.best_params_

{'learning_rate': 0.01}

In [87]:
xgb_reg_model.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=123,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [88]:
predictions_f = xgb_reg_model.predict(predict_X)

### M Estimate

In [89]:
xgb_reg_model.fit(X, y_m)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=123,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_param

In [90]:
xgb_reg_model.best_score_

-1.2471065310400011

In [91]:
xgb_reg_model.best_params_

{'learning_rate': 0.1}

In [92]:
xgb_reg_model.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=123,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [93]:
predictions_m= xgb_reg_model.predict(predict_X)

## Combine Predictions with New Customer Data

In [94]:
df_new_customer['r_score'] = predictions_r
df_new_customer['f_score'] = predictions_f
df_new_customer['m_score'] = predictions_m

df_new_customer

,customer_id,past_3_years_bike_related_purchases,tenure,property_valuation,age,0,1,2,3,4,5,6,7,8,9,10,r_score,f_score,m_score
0,10000,86.0,14.0,6.0,60.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.778007,1.327837,1.623520
1,10001,69.0,16.0,11.0,47.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.207044,1.406771,2.150128
2,10002,10.0,10.0,5.0,43.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.114688,1.271321,1.682933
3,10003,64.0,5.0,1.0,38.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.583416,1.347173,1.800781
4,10004,34.0,19.0,9.0,52.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.263505,1.430169,2.227696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,10995,60.0,9.0,7.0,58.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.934140,1.462908,2.499507
979,10996,22.0,6.0,10.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.794022,1.505905,3.424894
980,10997,17.0,15.0,2.0,63.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.622069,1.291041,1.636723
981,10998,30.0,19.0,2.0,65.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.797718,2.080430,3.528145


In [95]:
df_new_customer_list

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,customer_id
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,10000
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,10001
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,10002
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,10003
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,10004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,9,2 Sloan Way,2200,NSW,Australia,7,10995
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,10996
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,15,33475 Fair Oaks Junction,4702,QLD,Australia,2,10997
998,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,19,57666 Victoria Way,4215,QLD,Australia,2,10998


In [96]:
df_new_customer_list = df_new_customer_list.merge(df_new_customer[['customer_id', 'r_score', 'f_score', 'm_score']], on='customer_id', how='left')
df_new_customer_list

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,customer_id,r_score,f_score,m_score
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,10000,1.778007,1.327837,1.623520
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,10001,2.207044,1.406771,2.150128
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,10002,1.114688,1.271321,1.682933
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,10003,1.583416,1.347173,1.800781
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,10004,2.263505,1.430169,2.227696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,9,2 Sloan Way,2200,NSW,Australia,7,10995,1.934140,1.462908,2.499507
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,10996,2.794022,1.505905,3.424894
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,15,33475 Fair Oaks Junction,4702,QLD,Australia,2,10997,2.622069,1.291041,1.636723
998,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,19,57666 Victoria Way,4215,QLD,Australia,2,10998,2.797718,2.080430,3.528145


In [97]:
df_new_customer_list[['r_score', 'f_score', 'm_score']].describe()

,r_score,f_score,m_score
count,983.000000,983.000000,983.000000
mean,2.144511,1.461185,2.267854
std,0.407104,0.273510,0.528666
min,0.171759,0.529834,0.730493
25%,1.912668,1.355122,1.941821
50%,2.152059,1.411359,2.219576
75%,2.387978,1.462977,2.523290
max,3.638783,2.415042,4.241095


In [98]:
# New Customers with the highest potential recency
df_new_customer_list[df_new_customer_list['r_score'] >= 2.38]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,customer_id,r_score,f_score,m_score
8,Duff,Karlowicz,Male,50,1972-04-28,Speech Pathologist,Manufacturing,Mass Customer,N,Yes,5,717 West Drive,2200,NSW,Australia,10,10008,2.457709,1.453401,2.504591
9,Barthel,Docket,Male,72,1985-08-02,Accounting Assistant IV,IT,Mass Customer,N,Yes,17,80 Scofield Junction,4151,QLD,Australia,5,10009,2.431511,1.405092,2.159261
22,Otis,Ottey,Male,26,1998-02-05,Quality Engineer,NaN,Mass Customer,N,No,3,1562 Merchant Street,4744,QLD,Australia,4,10022,2.810521,1.457385,2.527245
25,Rourke,Gillbard,Male,11,1945-08-03,NaN,Property,Mass Customer,N,No,17,75 Cordelia Trail,4817,QLD,Australia,4,10025,2.684296,2.409967,3.746588
26,Dyane,Burwell,Female,19,1952-06-27,Engineer II,Retail,Mass Customer,N,Yes,7,51 Hooker Court,2640,NSW,Australia,2,10026,2.457154,1.596271,1.319220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,Lolly,Prewer,Female,44,1997-10-23,Geologist I,Health,Mass Customer,N,No,15,694 Coolidge Center,2620,NSW,Australia,7,10987,2.414256,1.497031,2.561810
993,Weidar,Etheridge,Male,38,1959-07-13,Compensation Analyst,Financial Services,Mass Customer,N,Yes,6,0535 Jay Point,2422,NSW,Australia,4,10993,2.569948,1.446216,2.543978
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,10996,2.794022,1.505905,3.424894
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,15,33475 Fair Oaks Junction,4702,QLD,Australia,2,10997,2.622069,1.291041,1.636723


In [99]:
# New Customers with the highest potential frequency
df_new_customer_list[df_new_customer_list['f_score'] >= 1.46]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,customer_id,r_score,f_score,m_score
5,Druci,Brandli,Female,39,1951-04-29,Assistant Media Planner,Entertainment,High Net Worth,N,Yes,22,89105 Pearson Terrace,4075,QLD,Australia,7,10005,2.039722,1.831141,3.194257
10,Rockwell,Matson,Male,94,1995-01-01,Programmer Analyst I,Retail,High Net Worth,N,No,3,3682 Crowley Point,4573,QLD,Australia,6,10010,1.977872,1.823745,2.832402
18,Odilia,Quick,Female,65,1938-11-09,General Manager,Manufacturing,Affluent Customer,N,Yes,11,1550 Russell Way,2222,NSW,Australia,11,10018,2.266264,2.121295,2.912225
19,Karly,Willavize,Female,2,1954-08-12,Internal Auditor,Manufacturing,High Net Worth,N,No,12,193 North Point,2190,NSW,Australia,10,10019,2.223803,1.858884,2.399981
25,Rourke,Gillbard,Male,11,1945-08-03,NaN,Property,Mass Customer,N,No,17,75 Cordelia Trail,4817,QLD,Australia,4,10025,2.684296,2.409967,3.746588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,Lolly,Prewer,Female,44,1997-10-23,Geologist I,Health,Mass Customer,N,No,15,694 Coolidge Center,2620,NSW,Australia,7,10987,2.414256,1.497031,2.561810
988,Vyky,Pegg,Female,64,1943-03-15,Quality Control Specialist,Manufacturing,Mass Customer,N,Yes,11,31854 Anniversary Terrace,2322,NSW,Australia,7,10988,1.842588,2.110831,3.533679
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,9,2 Sloan Way,2200,NSW,Australia,7,10995,1.934140,1.462908,2.499507
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,10996,2.794022,1.505905,3.424894


In [100]:
# New Customers with the highest potential spend
df_new_customer_list[df_new_customer_list['m_score'] >= 2.52]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,customer_id,r_score,f_score,m_score
5,Druci,Brandli,Female,39,1951-04-29,Assistant Media Planner,Entertainment,High Net Worth,N,Yes,22,89105 Pearson Terrace,4075,QLD,Australia,7,10005,2.039722,1.831141,3.194257
10,Rockwell,Matson,Male,94,1995-01-01,Programmer Analyst I,Retail,High Net Worth,N,No,3,3682 Crowley Point,4573,QLD,Australia,6,10010,1.977872,1.823745,2.832402
11,Wheeler,Winward,Male,48,1999-08-30,Environmental Specialist,Manufacturing,Mass Customer,N,No,10,3 Golden Leaf Point,3216,VIC,Australia,8,10011,2.228384,1.441571,2.545220
18,Odilia,Quick,Female,65,1938-11-09,General Manager,Manufacturing,Affluent Customer,N,Yes,11,1550 Russell Way,2222,NSW,Australia,11,10018,2.266264,2.121295,2.912225
22,Otis,Ottey,Male,26,1998-02-05,Quality Engineer,NaN,Mass Customer,N,No,3,1562 Merchant Street,4744,QLD,Australia,4,10022,2.810521,1.457385,2.527245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,Lolly,Prewer,Female,44,1997-10-23,Geologist I,Health,Mass Customer,N,No,15,694 Coolidge Center,2620,NSW,Australia,7,10987,2.414256,1.497031,2.561810
988,Vyky,Pegg,Female,64,1943-03-15,Quality Control Specialist,Manufacturing,Mass Customer,N,Yes,11,31854 Anniversary Terrace,2322,NSW,Australia,7,10988,1.842588,2.110831,3.533679
993,Weidar,Etheridge,Male,38,1959-07-13,Compensation Analyst,Financial Services,Mass Customer,N,Yes,6,0535 Jay Point,2422,NSW,Australia,4,10993,2.569948,1.446216,2.543978
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,10996,2.794022,1.505905,3.424894
